In [2]:
import pandas as pd
import sys
import csv
from statistics import median

In [3]:
project_ytest_lib = {}

In [4]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [19]:
def get_best_confidence(data):
    
    #typecasting
    data['project_reqd_builds'] = pd.to_numeric(data['project_reqd_builds'], downcast="float", errors='coerce')   
    
    #setting parameters
    project_list = set(data['project'].tolist())
    batching_alg = set(data['algorithm'].tolist())
    batchsizes = [1,2,4,8,16]
    
    #result file
    result_file = open('best_confidence.csv', 'w')
    res_headers = ['project', 'method', 'algorithm', 'batch_size', 'builds_saved', 'testall_size', 'confidence']
    res_writer = csv.writer(result_file)
    res_writer.writerow(res_headers)
    
    
    #get best confidences

    for p in project_list:
        
        p_name = p.split('/')[1]
        alg_scores = {}
        p_data = data[data['project'] == p]
        
        y_test = pd.read_csv('../data/25_1_travis_data/' + p)['tr_status'].tolist()
        y_test = output_values(y_test)
        
        for alg in batching_alg:

            alg_data = p_data[p_data['algorithm'] == alg]

            for b in batchsizes:

                if alg == 'BATCH4':
                    if b != 4:
                        continue

                if alg == 'BATCHSTOP4':
                    if b < 4:
                        continue
                        

                batch_data = alg_data[ alg_data['batch_size'] == b]
                
                total = batch_data['testall_size'].tolist()[0]
                reqd_list = batch_data['project_reqd_builds'].tolist()
                
                min_distance = sys.maxsize

                best = [p_name, 'ssr', alg, b, 0, 0, 0]
                best_i = 0

                for i in range(len(reqd_list)):
                    distance = reqd_list[i]
                    if distance < min_distance:
                        best[4] = 100-reqd_list[i]
                        best[5] = total
                        best[6] = i+2
                        best_i = i
                        min_distance = distance

                res_writer.writerow(best)
    

In [20]:
data = pd.read_csv('combined_results.csv')

In [21]:
get_best_confidence(data)

92

In [24]:
result_file = open('sbs_delay_lists.csv', 'w')
res_headers = ['project', 'method', 'algorithm', 'batch_size', 'median_sbs_delays', 'median_batch_delays', 'confidence']
res_writer = csv.writer(result_file)
res_writer.writerow(res_headers)

86

In [25]:
lines = []

In [26]:
for p in project_list:
    p_name = p + '.csv'
    alg_scores = {}

    p_data = data[data['project'] == p]
    #print(p_data)
    for alg in batching_alg:

        alg_data = p_data[p_data['algorithm'] == alg]
        #print(alg_data)

        for b in batchsizes:

            if alg == 'BATCH4':
                if b != 4:
                    continue

            if alg == 'BATCHSTOP4':
                if b < 4:
                    continue

            batch_data = alg_data[ alg_data['batch_size'] == b]

            reqd_list = batch_data['project_reqd_builds'].tolist()
            delay_list = batch_data['total_sbs_delay'].tolist()
            batch_delay_list = batch_data['total_batch_delay'].tolist()
            ssr_delays = batch_data['project_delays'].tolist()
            min_distance = sys.maxsize

            best = [p_name, 'ssr', alg, b, 0, 0]
            best_i = 0

            for i in range(len(reqd_list)):
                distance = ((reqd_list[i]**2)+((delay_list[i]+batch_delay_list[i])**2))**0.5
                if distance < min_distance:
                    best[4] = ssr_delays[i]
                    #best[5] = median(ssr_delays[i])
                    best[5] = i+2
                    best_i = i
                    min_distance = distance

            lines.append(best)
            if alg == 'BATCH4':
                print(best)

['cloudify.csv', 'ssr', 'BATCH4', 4, '[]', 189]


In [27]:
ssr_delay = pd.DataFrame(lines, columns=['project', 'method', 'algorithm', 'batch_size', 'sbs_delays', 'confidence'])

In [28]:
ssr_delay.to_csv('sbs_delay_list.csv')